# Data Loading Module

The present notebook reviews the *dataloading* module from the package *DL* of the library *QQuantLib*  (**QQuantLib/DL/data_loading.py**). 

This module deals with loading the data into the quantum states (or circuits). 

The present notebook and module are based on the following references:

* *Grover, Lov and Rudolph, Terry*. Creating superposition that correspond to efficiently integrable probability distributions. arXiv (2002). https://arxiv.org/abs/quant-ph/0208112

* *V.V. Shende, S.S. Bullock, and I.L. Markov*. Synthesis of quantum-logic circuits. IEEE Transactions on Computer-Aided Design of Integrated Circuits and Systems, 25(6):1000–1010, Jun 2006. https://arxiv.org/abs/quant-ph/0406176v5

* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
#This cell loads the QLM solver. QPU = [qlmass, python, c]
from QQuantLib.utils.get_qpu import get_qpu
QPU = ["qlmass", "python", "c"]
linalg_qpu = get_qpu(QPU[2])

The following cell loads the *get_results* function from **QQuantLib/utils/data_extracting**. The present library uses this function frequently so here we explain the use, the input and the outputs.

*get_results* function receives a QLM object (a QLM gate, QRoutine or Program), and creates the associated QLM program, circuit and job. Additionally runs the QLM job (simulates the quantum program), gets the results and post-processes them.

Inputs for *get_results*:

* quantum_object : QLM Gate, Routine or Program
* linalg_qpu : QLM solver
* shots : int (number of shots for the generated job)
* qubits : list (list with the qubits for doing the measurement when simulating)
* complete : bool. For returning the complete basis state. Useful when shots are not 0 and all the possible basis states are required

The outputs of *get_results* will be:
* pdf : pandas DataFrame. DataFrame with the results of the simulation
* circuit : QLM circuit of the QLM input object
* q_prog : QLM Program of the QLM input object
* job : QLM job of the QLM input object

The main output of this function is the pandas DataFrame. The columns provided in the DataFrame are:

* **States**: Possible quantum states of the measurements done on the circuit
* **Int_lsb**: conversion from the quantum state to an integer following **lsb** (bit farthest to the right will be least significant)
* **Probability**: Computed frequency of the quantum state when *shots* is not zero. When *shots* is zero computed probabilities are providing
* **Amplitude**: Amplitude of the quantum states. Only providing if *shots* is zero.
* **Int**: conversion from the quantum state to an integer (the bit farthest to the right will be most significant)


In [ ]:
from QQuantLib.utils.data_extracting import get_results

## 1. Loading Data

Typically, when we want to load some data into the quantum circuit, we will want to load a discrete probability distribution $p_d$ and an array $f$. First thing we need to define the dimension of the circuit and what we want to load. Here $n$ is the number of qubits and $N = 2^n$ is the size of the discretized probability distribution and the size of the array. In this specific example $n = 3$ and $N = 8$.

In [ ]:
n = 3
N = 2**n
x = np.arange(N)

Next, we define a discrete probability distribution:

$$p_d = \left(p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7\right).$$

In this specific example we are going to generate the following probability distribution:

$$p_d = \dfrac{1}{0+1+2+3+4+5+6+7}\left(0,1,2,3,4,5,6,7\right),$$
which is saved in the variable *probability*.

In [ ]:
probability = x/np.sum(x)

Finally, we define an array:
$$f = \left(f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7\right).$$
Later, it will become useful to have a normalised version of this function that we will call $\hat{f}$. This new function $\hat{f}$ has the main characteristic that where the maximum absolute value of the function is one $||\hat{f}||_{\infty} = 1$:
$$\hat{f} = \dfrac{f}{||f||_{\infty}} = \left(\hat{f}_0,\hat{f}_1,\hat{f}_2,\hat{f}_3,\hat{f}_4,\hat{f}_5,\hat{f}_6,\hat{f}_7\right).$$
In the code, this is the reason why we introduce the variable *normalization_constant* $=||f||_{\infty}$.

In this specific example, we choose $f$ to simply be:
$$f = \left(0,1,2,3,4,5,6,7\right).$$
Hence, $\hat{f}$ is:
$$\hat{f} = \dfrac{f}{||f||_{\infty}} = \dfrac{1}{7}\left(0,1,2,3,4,5,6,7\right).$$


In [ ]:
normalization_constant = np.max(x)
f = x
f_normalised = x/normalization_constant

In [ ]:
%matplotlib inline
plt.plot(x, probability, 'o')
plt.plot(x, f_normalised, 'o')
plt.grid()
plt.legend(['Probability', 'Array to load'])

### 1.1 Loading Probability

In [ ]:
from QQuantLib.DL.data_loading import load_probability

To load a discrete probability distribution we just need the function *load_probability*, which is inside the **DL/data_loading** module. The inputs are:

* numpy array with the probability distribution that we want to load into the quantum state (**MANDATORY**). In this case, the probability distribution is the variable *probability*.
* id_name: string for giving a name to the Abstract Gate created by the function. If no name is provided then the CPU time will be added to the name.
* method: string for selecting the internal implementation for the controlled rotation by state (which is the basis of our data loading method). Two possible values:
    * multiplexor: using quantum multiplexors for the data loading. This is the default value.
    * brute_force: using a direct implementation of the controlled rotation by state (longer circuits)

The output of the function is a **qlm** *AbstractGate* with arity *n*. 

In [ ]:
#No id_name provided
routine = load_probability(probability)
#Display circuit representation
%qatdisplay routine --depth 0 --svg

Now, our quantum state has the form:

$$\left[\sqrt{p_0}|0\rangle+\sqrt{p_1}|1\rangle+\sqrt{p_2}|2\rangle+\sqrt{p_3}|3\rangle+\sqrt{p_4}|4\rangle+\sqrt{p_5}|5\rangle+\sqrt{p_6}|6\rangle+\sqrt{p_7}|7\rangle\right]$$

Last, we use the function *get_results* from **data_extracting** to obtain the probabilities loaded into the quantum circuit. By the quantum properties, what we are measuring is:


$$\left(\sqrt{p_0}^2,\sqrt{p_1}^2,\sqrt{p_2}^2,\sqrt{p_3}^2,\sqrt{p_4}^2,\sqrt{p_5}^2,\sqrt{p_6}^2,\sqrt{p_7}^2\right) = \left(p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7\right)$$

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values
print("Quantum probabilities: ",quantum_probabilities)
print("Classical probabilities: ",probability)
#Test
np.isclose(quantum_probabilities, probability).all()

### brute-force implementation

In [ ]:
#With id_name provided
routine_bf = load_probability(probability, id_name='brute_force', method = 'brute_force')
#Display circuit representation
%qatdisplay routine_bf --depth  --svg

In [ ]:
results,_,_,_ = get_results(routine_bf, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values
print("Quantum probabilities: ",quantum_probabilities)
print("Classical probabilities: ",probability)
#Test
np.isclose(quantum_probabilities, probability).all()

### quantum multiplexors implementation

In [ ]:
#With id_name provided
routine_mp = load_probability(probability, id_name='multiplexors')
#Display circuit representation
%qatdisplay routine_mp --depth  --svg

In [ ]:
results,_,_,_ = get_results(routine_mp, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values
print("Quantum probabilities: ",quantum_probabilities)
print("Classical probabilities: ",probability)
#Test
np.isclose(quantum_probabilities, probability).all()

As can be seen the Quantum multiplexors implementation is lower in depth.

In order to work properly, the function *load_probabilities* has to be the first gate of the whole circuit.

### 1.2 Loading Function

To load an array we need more steps than in the previous example. We first need to load a probability distribution in the state and reserve an extra qubit for loading the function. Here we don't want to focus on the part of loading a probability distribution as it has been already treated in the previous subsection, for that reason we will simply call the function *uniform_distribution* **DL/data_loading** module

In [ ]:
from QQuantLib.DL.data_loading import uniform_distribution

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
routine.apply(uniform_distribution(n),register[:n])

Our circuit has the form:

In [ ]:
%qatdisplay routine --depth 1 --svg

The state now loaded in the circuit is:

$$\dfrac{1}{\sqrt{N}}|0\rangle\left[|0\rangle+|1\rangle+|2\rangle+|3\rangle+|4\rangle+|5\rangle+|6\rangle+|7\rangle\right]$$

The next step is loading our array.  For that we have the function *load_array* inside **DL/data_loading** module, this function takes following arguments:

1. Normalised array, one which has norm infinity equal to or less than $1$. 
2. Argument called *method*. By default, this second argument is set to *multiplexors*, but it can also have the value *brute_force*. This second option is much less efficient in terms of quantum gates.
3. id_name: string for giving a name to the Abstract Gate created by the function. If no name is provided then the CPU time will be added to the name.

In [ ]:
from QQuantLib.DL.data_loading import load_array

In [ ]:
#Not Providing id_name
load_gate = load_array(f_normalised)
%qatdisplay load_gate --depth 0 --svg

In [ ]:
#Providing id_name
load_gate = load_array(f_normalised, id_name='f')
%qatdisplay load_gate --depth 0 --svg

In [ ]:
#Now apply the gate to the routine
routine.apply(load_gate, register)


$$\dfrac{1}{\sqrt{N}}|0\rangle\left[\hat{f}_0|0\rangle+\hat{f}_1|1\rangle+\hat{f}_2|2\rangle+\hat{f}_3|3\rangle+\hat{f}_4|4\rangle+\hat{f}_5|5\rangle+\hat{f}_6|6\rangle+\hat{f}_7|7\rangle\right]+...$$
The associated circuit is:

In [ ]:
%qatdisplay routine --depth 0 --svg

Finally, we measure the probabilities that we have loaded.

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

By the quantum properties, the result stored in the variable *quantum_probabilities* is the absolute value of the square of $\hat{f}$ divided by $N$, that is:
$$p = \dfrac{1}{N}\left(\hat{f}_0^2,\hat{f}_1^2,\hat{f}_2^2,\hat{f}_3^2,\hat{f}_4^2,\hat{f}_5^2,\hat{f}_6^2,\hat{f}_7^2,...\right)$$
There is more information stored in the variable, but we don't need it.

If we want to recover the function $f$ we need to compute the square root of the probabilities and multiplicate by $N$ and by the normalization constant:
$$f_i = N||f||_{\infty}\sqrt{p_i}$$

In [ ]:
quantum_f = np.sqrt(quantum_probabilities)*np.sqrt(N)*normalization_constant
print("Array loaded in the quantum state: ",quantum_f[:N])
print("Original array: ",f)
#Test
np.isclose(quantum_f[:N], f).all()

### 1.3 Loading a function upon a non trivial probability distribution

Now we are going to load a discrete probability distribution $p_d$ and an array $f$ altogether. This case is a combination of the two previous cases. First, we start by loading the discrete probability distribution $p_d$. Note that, for loading the normalised array $\hat{f}$ we need an extra qubit. The probability function is loaded just in the first three registers: 


In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
routine.apply(load_probability(probability, id_name='p(x)'),register[:n])
%qatdisplay routine --depth 0 --svg

Now our quantum state is:
$$|0\rangle\left[\sqrt{p_0}|0\rangle+\sqrt{p_1}|1\rangle+\sqrt{p_2}|2\rangle+\sqrt{p_3}|3\rangle+\sqrt{p_4}|4\rangle+\sqrt{p_5}|5\rangle+\sqrt{p_6}|6\rangle+\sqrt{p_7}|7\rangle\right]$$

Next, we compute the angles and load the function. Instead of loading $\hat{f}$ we are going to load $\sqrt{\hat{f}}$ to have everything at the same place.

In [ ]:
f_root = np.sqrt(f_normalised)
routine.apply(load_array(f_root, id_name='f(x)'),register)
%qatdisplay routine --depth 0 --svg

Now our quantum state is:
$$|0\rangle\left[\sqrt{p_0\hat{f}_0}|0\rangle+\sqrt{p_1\hat{f}_1}|1\rangle+\sqrt{p_2\hat{f}_2}|2\rangle+\sqrt{p_3\hat{f}_3}|3\rangle+\sqrt{p_4\hat{f}_4}|4\rangle+\sqrt{p_5\hat{f}_5}|5\rangle+\sqrt{p_6\hat{f}_6}|6\rangle+\sqrt{p_7\hat{f}_7}|7\rangle\right]+...$$
If we measure again, we can compare our result with the element-wise product of $p_d$ and $f$

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

In [ ]:
quantum_result = quantum_probabilities*normalization_constant
print("Quantum result: ",quantum_result[0:N])
print("Classical result: ",probability*f)
np.isclose(quantum_result[0:N], probability*f).all()

If we wanted to compute the scalar product from the previous technique, we can use a neat trick. If we just measure the last qubit (the one that is more on the left in the state or the one that is at the bottom of the circuit) we are effectively computing this amount:

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu,qubits = [n])
quantum_probabilities = results["Probability"].values

In [ ]:
quantum_result = quantum_probabilities*normalization_constant
print("Quantum result: ",quantum_result[0])
print("Classical result: ",np.dot(probability,f))
#Test
np.isclose(quantum_result[0], np.dot(probability,f))

### 1.4 Loading two arrays

In our final example, we are going to load two arrays $f$ and $g = p_d$. To load two arrays we need two extra qubits, one for the first array and another for the second one. We start again by defining our base routine with size $n+2$. As we always need to load a base distribution we load a uniform probability distribution.

In [ ]:
g = probability
routine = qlm.QRoutine()
register = routine.new_wires(n+2)
routine.apply(uniform_distribution(n),register[:n])
%qatdisplay routine --depth 0 --svg

We already have defined the normalised version of $f$, $\hat{f}$ which is stored in the variable *f_normalised*. However the discrete probability distribution $p_d$ is not normalised. For this reason, we define the normalised version as:
$$\hat{g} = \dfrac{g}{||g||_{\infty}}.$$

In [ ]:
g_normalised = g/np.max(g)

Now we have to load the normalised arrays in the circuit. Because we have two arrays instead of one, we are going to load the first array in the first reserved register and the second one in the second reserved register.

In [ ]:
routine.apply(load_array(f_normalised,id_name= '1'),register[0:n],register[n])
routine.apply(load_array(g_normalised,id_name= '2'),register[0:n],register[n+1])

In [ ]:
%qatdisplay routine --depth 0 --svg

Now our quantum state is:
$$\dfrac{1}{\sqrt{N}}|0\rangle|0\rangle\left[f_0 g_0|0\rangle+f_1 g_1|1\rangle+f_2 g_2|2\rangle+f_3 g_3|3\rangle+f_4 g_4|4\rangle+f_5 g_5|5\rangle+f_6 g_6|6\rangle+f_7 g_7|7\rangle\right]+...$$
If we measure again, we can compare our result with the element-wise product of $g$ and $f$

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

In [ ]:
quantum_result = np.sqrt(quantum_probabilities*N)*np.max(f)*np.max(g)
classical_result = f*g
print("Quantum result: ",quantum_result[:N])
print("Classical result: ",classical_result)
np.isclose(quantum_result[:N], classical_result).all()

If we want to compute the scalar product from our state we simply need to do a Hadamard transform. The first coefficient of the Hadamard transform is the sum of the input vector, in other words, our previous state:

$$\dfrac{1}{\sqrt{N}}|0\rangle|0\rangle\left[f_0 g_0|0\rangle+f_1 g_1|1\rangle+f_2 g_2|2\rangle+f_3 g_3|3\rangle+f_4 g_4|4\rangle+f_5 g_5|5\rangle+f_6 g_6|6\rangle+f_7 g_7|7\rangle\right]+...,$$
transforms to:
$$\dfrac{1}{N}|0\rangle|0\rangle\left[f_0 g_0+f_1 g_1+f_2 g_2+f_3 g_3+f_4 g_4+f_5 g_5+f_6 g_6+f_7 g_7\right|0\rangle+...]+...$$
The rest of the coefficients depend on which specific transformation we are doing, as there are three different versions of this transformation. Note that instead of a factor $\dfrac{1}{\sqrt{N}}$ now we have a factor $\dfrac{1}{N}$.

In [ ]:
routine.apply(uniform_distribution(n),register[:n])
%qatdisplay routine --depth 0 --svg

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

In [ ]:
quantum_result = np.sqrt(quantum_probabilities)*N*np.max(f)*np.max(g)
classical_result = np.dot(f,g)
print("Quantum result: ",quantum_result[0])
print("Classical result: ",classical_result)
#Test
np.isclose(quantum_result[0], classical_result)

### 1.5 Ordering conventions

Here we are following the ordering convention $|q_n...q_1q_0\rangle$ with $q_0$ being the least significant qubit. However, in plain QLM they follow the opposite convention where $|q_0q_1...q_n\rangle$. Here we show how to use our functions to follow the QLM convention, indeed it is not difficult.

First, for loading a distribution we just need to apply the loading gate in opposite order:

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n)
routine.apply(load_probability(probability),register[::-1])
%qatdisplay routine --depth 1 --svg

To get the results we do the same as always.

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)

Now we use the QLM ordering to get the probabilities: for this we can use the *Int* column from the *results* DataFrame

In [ ]:
quantum_probabilities = results.sort_values("Int")["Probability"].values
print("Quantum probabilities: ",quantum_probabilities)
print("Classical probabilities: ",probability)
np.isclose(quantum_probabilities, probability).all()

If we have to deal with the loading of functions it can be a little bit more tricky:

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
control = register[:n]
target = register[n]
routine.apply(uniform_distribution(n),register[:n])
routine.apply(load_array(f_normalised),[control[::-1],target])
%qatdisplay routine --depth 0 --svg

To retrieve the results it is less convenient as now the result is not in order 

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
results.sort_values("Int",inplace = True)
results["Function"] = np.sqrt(results["Probability"]*N)*normalization_constant
results

See that now the information is stored in all qubits marked with a zero in the leftmost qubit. That provoques that the information is stored only in the odd positions (with respect to the int convention).
Another possibility is applying everything in reverse:

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
control = register[1:n+1]
target = register[0]
routine.apply(uniform_distribution(n),register[1:n+1])
routine.apply(load_array(f_normalised),[control[::-1],target])
%qatdisplay routine --depth 0 --svg

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
results.sort_values("Int",inplace = True)
results["Function"] = np.sqrt(results["Probability"]*N)*normalization_constant
results

In this case the results are naturally stored in the order of QLM (the int order). 

## Appendix 1: Loading array with angles

When we load an array, under the hood we are really loading some angles into the quantum circuit. In this appendix, we show how to use the angles to load a properly normalised array $\hat{f}$. As always we start by loading a base discrete probability distribution.

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
routine.apply(uniform_distribution(n),register[:n])

 If we want to load the array $\hat{f}$ using angles, we first have to compute the associated angles:
$$ \left(\theta_0,\theta_1,\theta_2,\theta_3,\theta_4,\theta_5,\theta_6,\theta_7\right)= \theta =2\arccos\left(\hat{f}\right)=\left(2\arccos\left(\hat{f}_0\right),2\arccos\left(\hat{f}_1\right),2\arccos\left(\hat{f}_2\right),2\arccos\left(\hat{f}_3\right),2\arccos\left(\hat{f}_4\right),2\arccos\left(\hat{f}_5\right),2\arccos\left(\hat{f}_6\right),2\arccos\left(\hat{f}_7\right)\right)$$
This is the reason why we need to work with the normalised version of function $f$.

In [ ]:
angles = 2*np.arccos(f_normalised)

To load angles we will use the function **load_angles**, which is inside the **data_loading** module. The input should be a numpy array with the angles associated to the normalised function $\hat{f}$. In this case, the probability distribution is the variable *probability*. The output of the function is a **qlm** *AbstractGate* with arity *n*.

Next, we load the angles into the quantum state. For that, we have the function *load_angles*, these function admits a second argument called *method*. By default, it is set to *multiplexors*, but it can also have the value *brute_force*. The second option is much less efficient in terms of quantum gates.

In [ ]:
from QQuantLib.DL.data_loading import load_angles

In [ ]:
routine.apply(load_angles(angles),register)

Now, our quantum state has the form:
$$\dfrac{1}{\sqrt{N}}|0\rangle\left[\cos\left(\dfrac{\theta_0}{2}\right)|0\rangle+\cos\left(\dfrac{\theta_1}{2}\right)|1\rangle+\cos\left(\dfrac{\theta_2}{2}\right)|2\rangle+\cos\left(\dfrac{\theta_3}{2}\right)|3\rangle+\cos\left(\dfrac{\theta_4}{2}\right)|4\rangle+\cos\left(\dfrac{\theta_5}{2}\right)|5\rangle+\cos\left(\dfrac{\theta_6}{2}\right)|6\rangle+\cos\left(\dfrac{\theta_7}{2}\right)|7\rangle\right]+...,$$
substituting the value of the angles we have the state
$$\dfrac{1}{\sqrt{N}}|0\rangle\left[\hat{f}_0|0\rangle+\hat{f}_1|1\rangle+\hat{f}_2|2\rangle+\hat{f}_3|3\rangle+\hat{f}_4|4\rangle+\hat{f}_5|5\rangle+\hat{f}_6|6\rangle+\hat{f}_7|7\rangle\right]+...$$
The associated circuit is:

In [ ]:
%qatdisplay routine --depth 0 --svg

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

Finally, we measure the probabilities that we have loaded and see that is the same as using the function **load_array**

In [ ]:
quantum_f = np.sqrt(quantum_probabilities)*np.sqrt(N)*normalization_constant
print("Array loaded in the quantum state: ",quantum_f[:N])
print("Original array: ",f)
np.isclose(quantum_f[:N], f).all()

## Appendix 2: Quantum Multiplexors

Implementation of data loading routines using the *Lov Grover and Terry Rudolph* routines directly, using controlled rotations by state, is highly inefficient. In general, the use of controlled rotations generates highly deep quantum circuits prone to errors. A more efficient approach is the use of Quantum Multiplexors.

A Quantum Multiplexor in our case is a routine that applies a quantum gate depending on a control register. To simplify things, we are interested in Quantum Multiplexed Ry gates. Specifically, a Quantum Multiplexed Ry gate applies a $Ry$ rotation to a specific qubit depending on a control register. 
To give an example we are going to focus on a register with four qubits, the first three ones are the controls and the fourth one is the target qubit:
$$
\begin{array}{l}
&|0000\rangle\longrightarrow |0\rangle|0\rangle\\
&|0001\rangle\longrightarrow |0\rangle|1\rangle\\
&|0010\rangle\longrightarrow |0\rangle|2\rangle\\
&|0011\rangle\longrightarrow |0\rangle|3\rangle\\
&|0100\rangle\longrightarrow |0\rangle|4\rangle\\
&|0101\rangle\longrightarrow |0\rangle|5\rangle\\
&|0110\rangle\longrightarrow |0\rangle|6\rangle\\
&|0111\rangle\longrightarrow |0\rangle|7\rangle\\
\end{array}
$$
A quantum Quantum Multiplexed Ry gate applies a $Ry$ gate to the target register with angles $(\theta_0,\theta_1,\theta_2,\theta_3,\theta_4,\theta_5,\theta_6,\theta_7)$. It is easier to visualize it:
$$
\begin{array}{l}
&|0\rangle|0\rangle\longrightarrow \cos(\theta_0)|0\rangle|0\rangle+\sin(\theta_0)|1\rangle|0\rangle\\
&|0\rangle|1\rangle\longrightarrow \cos(\theta_1)|0\rangle|1\rangle+\sin(\theta_1)|1\rangle|1\rangle\\
&|0\rangle|2\rangle\longrightarrow \cos(\theta_2)|0\rangle|2\rangle+\sin(\theta_2)|1\rangle|2\rangle\\
&|0\rangle|3\rangle\longrightarrow \cos(\theta_3)|0\rangle|3\rangle+\sin(\theta_3)|1\rangle|3\rangle\\
&|0\rangle|4\rangle\longrightarrow \cos(\theta_4)|0\rangle|4\rangle+\sin(\theta_4)|1\rangle|4\rangle\\
&|0\rangle|5\rangle\longrightarrow \cos(\theta_5)|0\rangle|5\rangle+\sin(\theta_5)|1\rangle|5\rangle\\
&|0\rangle|6\rangle\longrightarrow \cos(\theta_6)|0\rangle|6\rangle+\sin(\theta_6)|1\rangle|6\rangle\\
&|0\rangle|7\rangle\longrightarrow \cos(\theta_7)|0\rangle|7\rangle+\sin(\theta_7)|1\rangle|7\rangle\\
\end{array}
$$
This is the idea. Now, to implement this operator in the original paper they propose a recursive way. We prefer to propose a new method to do it without the need of recursion. In this way, it is easier to analyse the computational cost.

To do the implementation we just need to ingredients, CNOT gates and single rotations. The CNOT gates and the RY rotations have to be applied in an alternating order. See the image below.

In [ ]:
m = 3
angles = np.arange(2**m)
angles = angles/np.max(angles)
routine = qlm.QRoutine()
register = routine.new_wires(m+1)
routine.apply(load_angles(angles),register)
%qatdisplay routine --depth 0 --svg

See that the CNOT gates have always as a target the target register. The controls simply have $2^i$ periodicity with a certain $i$ depending on the position. In simpler words, the first CNOT (in the image above the one controlling in $q_2$, in the image below it controls) is in the positions $(0,2,4,6...)$. The second one is going to be in positions $(1+0,1+4,1+8,...)$. In the image below we can see a more extensive example:

In [ ]:
m = 4
angles = np.arange(2**m)
angles = angles/np.max(angles)
routine = qlm.QRoutine()
register = routine.new_wires(m+1)
routine.apply(load_angles(angles),register)
%qatdisplay routine --depth 0 --svg

Finally, the angles for the rotations are the Hadamard transform in sequence order of the angles that we want to load. For the circuit above it is possible to check it using the fast Walsh Hadamard transform:

In [ ]:
from QQuantLib.utils.utils import fwht

In [ ]:
fwht(angles)/2**m